# Question-Answer Pair Generation Using Large Language Models

## Overview
This notebook presents an innovative approach for generating high-quality question-answer pairs. Leveraging a Large Language Model (LLM), we aim to create a dataset that can be used for a variety of applications, particularly in fields requiring precise and accurate information retrieval, such as in medical contexts.

## Process Outline
Our approach is divided into four key steps:

### 1. Source Document Filtering
- **Objective**: To streamline the selection of relevant source material from a vast array of documents.
- **Method**: Implement a filtering mechanism that narrows down the number of source documents and selects pertinent paragraphs for subsequent processing.

### 2. Question Generation
- **Objective**: To formulate high-quality question candidates.
- **Method**: Utilize the capabilities of a Large Language Model, employing advanced prompt engineering techniques. Questions are designed to elicit responses that can either be direct extracts from selected source documents (PDFs) or concise, fact-based answers.

### 3. Automated Filtering of Q&A Pairs
- **Objective**: To refine the generated question-answer pairs and ensure their relevance and quality.
- **Methods**:
  - **N-gram Similarity**: Assess the similarity between the question and its context to filter out unrelated pairs.
  - **LLM Scoring**: Use a score provided by the LLM for the relatedness between the question and context.
  - **Text Embedding Vectors**: Create embeddings for questions to identify and remove highly similar questions.


In [1]:
!pip install langchain
!pip install openai
!pip install seaborn
!pip install matplotlib
!pip install pandas
!pip install scikit-learn
!pip install nltk
!pip install tiktoken


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\szond\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\szond\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load data

In [1]:
import json
with open("../scripts/datascraping/data/flyers_text.json", 'r') as file:
    data = json.load(file)

docs = []
for title, doc in data.items():
    for header, section in data.items():
        docs.append(str({title: {header: section}}))



## Source Document Filtering

In [5]:
import random
docs = [doc for doc in docs if len(doc) < 3000]
sample = random.sample(docs, 20)

sample

["{'Strikte isolatie': {'Sesquiterpene lactone mix': {'22. Sesquiterpene lactone mixWat is sesquiterpene lactone mix?': 'Deze mix bestaat uit 3 bestanddelen: \\n- Alantolactone; \\n- Costunolide; \\n- Dehydrocostunolide. Deze mix noemen we in de rest van de folder S-mix.Deze stoffen komen vooral voor bij composieten (bloemen). Dit zijn bijvoorbeeld duizendblad, wormkruid, valkruid, chrysant, kamille en moederkruid.', 'Wat zijn de verschijnselen van een SL-mix allergie?': 'Wanneer u allergisch bent voor deze stoffen krijgt u allergisch contacteczeem. Dit krijgt u vooral op de handen. Op lichaamsdelen, die niet bedekt zijn met kleding, kunt u ook eczeem krijgen door verwelkte plantendelen of soms door pollen. Gebruikt u waar deze stoffen in zijn gebruikt? Ook dan kunt u eczeem krijgen.', 'Wat kan ik doen tegen mijn SL-mix allergie?': 'Vermijd het contact met de bestanddelen van deze mix. Lees voor aanschaf van een product goed het etiket, controleer of er bestanddelen van deze mix in voo

## Question Generation

In [6]:
import pandas as pd
"""
from prompt_factory import PromptFactory
from question_generation import  QuestionGenerator
template = '''
        Gedraag je als een {role} die online informatie zoekt.
        Patiënten stellen meestal vragen als:
            "Mijn neusslang is eruit gevallen, wat moet ik doen?",
            "Wanneer mag ik niet in bad bevallen?"
        in het formaat: 
        {format}

        {document}
        
        Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het json formaat:
        {format}
        '''

roles = [
    'Patiënt',
    'Nieuwe Ouder',
    'Oudere Patiënt',
    'Persoon die een Tweede Mening Zoekt',
    'Reiziger die Medisch Advies Na Reizen Zoekt',
    'Zorgverlener die Informatie Zoekt']

prompt_factory = PromptFactory(prompt=template,roles=roles)

QuestionGenerator = QuestionGenerator()
results = [(doc, 
            QuestionGenerator.generate_questions(
                prompt_factory.generate_prompt(doc, 5))) 
           for doc in sample]
results
"""

'\nfrom prompt_factory import PromptFactory\nfrom question_generation import  QuestionGenerator\ntemplate = \'\'\'\n        Gedraag je als een {role} die online informatie zoekt.\n        Patiënten stellen meestal vragen als:\n            "Mijn neusslang is eruit gevallen, wat moet ik doen?",\n            "Wanneer mag ik niet in bad bevallen?"\n        in het formaat: \n        {format}\n\n        {document}\n        \n        Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het json formaat:\n        {format}\n        \'\'\'\n\nroles = [\n    \'Patiënt\',\n    \'Nieuwe Ouder\',\n    \'Oudere Patiënt\',\n    \'Persoon die een Tweede Mening Zoekt\',\n    \'Reiziger die Medisch Advies Na Reizen Zoekt\',\n    \'Zorgverlener die Informatie Zoekt\']\n\nprompt_factory = PromptFactory(prompt=template,roles=roles)\n\nQuestionGenerator = QuestionGenerator()\nresults = [(doc, \n            QuestionGenerator.generate_questions(\n                prompt_factory.generate_p

In [7]:
"""
from qa_generation import QAGenerator
from prompt_factory import PromptFactory

template = '''
        Gedraag je als een {role} die online informatie zoekt.
        Patiënten stellen meestal vragen als:
        {{
        "question": 
            "Hoe kan ik een verstopte voedingssonde doorspoelen?"
        "answer":
            "Om een verstopte
            voedingssonde door te
            spoelen, kunt u de volgende
            stappen proberen:
            1. Sluit de spuit
            rechtstreeks aan op
            de sonde (niet op het
            voedingssysteem).
            Als er een
            verstopping is in het
            voedingssysteem,
            kunt u dit vervangen.
            2. Neem een spuit van
            10 cc en spuit met
            lichte druk lauwwarm
            water door de
            voedingssonde.
            Herhaal dit zo nodig
            nog een keer.
            3. Als het oplossen van
            de verstopping niet
            lukt, laat dan
            lauwwarm water 30
            minuten inwerken en
            herhaal de
            bovenstaande
            procedure nogmaals.
            4. Als u de verstopping
            kunt zien, kunt u
            proberen om de
            voedingssonde op
            die plek zachtjes te
            kneden. Als de
            verstopping dan
            loskomt, kunt u het
            doorspuiten met
            lauwwarm water.
            Belangrijk: Gebruik nooit een
            voerdraad of
            koolzuurhoudend bronwater
            of frisdranken, omdat dit kan
            leiden tot perforatie.
            Daarnaast wordt het afraden
            om natriumbicarbonaat te
            gebruiken als medicatie de
            oorzaak is van de
            verstopping, omdat dit de
            verstopping groter kan
            maken."
        }}

        in het json formaat: 
        {format}

        {document}
        
        Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het formaat:
        {format}
        '''

roles = [
    'Patiënt',
    'Nieuwe Ouder',
    'Oudere Patiënt',
    'Persoon die een Tweede Mening Zoekt',
    'Reiziger die Medisch Advies Na Reizen Zoekt',
    'Zorgverlener die Informatie Zoekt']

prompt_factory = PromptFactory(prompt=template,roles=roles)

num_questions_per_doc = 5 

QAGenerator = QAGenerator()
# Generate the data
data = []
for doc in sample:
    for _ in range(num_questions_per_doc):
        try:
            qa = QAGenerator.generate_qa(prompt_factory.generate_prompt(doc, 1))
            data.append({'Document': doc, 'Question': qa.question, 'Answer': qa.answer})
        except Exception as e:
            print(f"Failed to generate QA for document: {doc}. Error: {e}")
            continue

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)
"""

'\nfrom qa_generation import QAGenerator\nfrom prompt_factory import PromptFactory\n\ntemplate = \'\'\'\n        Gedraag je als een {role} die online informatie zoekt.\n        Patiënten stellen meestal vragen als:\n        {{\n        "question": \n            "Hoe kan ik een verstopte voedingssonde doorspoelen?"\n        "answer":\n            "Om een verstopte\n            voedingssonde door te\n            spoelen, kunt u de volgende\n            stappen proberen:\n            1. Sluit de spuit\n            rechtstreeks aan op\n            de sonde (niet op het\n            voedingssysteem).\n            Als er een\n            verstopping is in het\n            voedingssysteem,\n            kunt u dit vervangen.\n            2. Neem een spuit van\n            10 cc en spuit met\n            lichte druk lauwwarm\n            water door de\n            voedingssonde.\n            Herhaal dit zo nodig\n            nog een keer.\n            3. Als het oplossen van\n            de ver

In [8]:
from qa_generation import QAGenerator
from prompt_factory import PromptFactory

template = '''
        Gedraag je als een {role} die online informatie zoekt.
        Patiënten stellen meestal vragen als:
{{
    "qa_list": [
        {{
            "question": "Hoe kan ik een verstopte voedingssonde doorspoelen?",
            "answer": "Om een verstopte voedingssonde door te spoelen, kunt u de volgende stappen proberen: 1. Sluit de spuit rechtstreeks aan op de sonde (niet op het voedingssysteem). Als er een verstopping is in het voedingssysteem, kunt u dit vervangen. 2. Neem een spuit van 10 cc en spuit met lichte druk lauwwarm water door de voedingssonde. Herhaal dit zo nodig nog een keer. 3. Als het oplossen van de verstopping niet lukt, laat dan lauwwarm water 30 minuten inwerken en herhaal de bovenstaande procedure nogmaals. 4. Als u de verstopping kunt zien, kunt u proberen om de voedingssonde op die plek zachtjes te kneden. Als de verstopping dan loskomt, kunt u het doorspuiten met lauwwarm water. Belangrijk: Gebruik nooit een voerdraad of koolzuurhoudend bronwater of frisdranken, omdat dit kan leiden tot perforatie. Daarnaast wordt het afraden om natriumbicarbonaat te gebruiken als medicatie de oorzaak is van de verstopping, omdat dit de verstopping groter kan maken."
        }}
    ]
}}


        in het json formaat: 
        {format}
        
Document Informatie:
        {document}
        
        Belangrijke Opmerking: Bij het genereren van vragen, gebruik specifieke termen en benamingen uit het document in plaats van algemene termen zoals 'dit onderzoek' of 'die procedure'. Verwijs direct naar de procedure of het document met de exacte naam om nauwkeurigheid en duidelijkheid in de vragen te waarborgen. Vermijd algemeenheden en zorg ervoor dat elke vraag direct gerelateerd is aan de verstrekte documentinformatie.

        
        Stel {n} vragen die beantwoord kunnen worden op basis van deze paragraaf in het formaat:
        {format}
        
        Zorg ervoor dat elke vraag en antwoord paar in een geldig JSON-formaat is. Dit betekent dat vragen en antwoorden tussen dubbele aanhalingstekens moeten staan, en de algemene structuur moet overeenkomen met het vereiste JSON-schema.
        '''

roles = [
    'Patiënt',
    'Nieuwe Ouder',
    'Oudere Patiënt',
    'Persoon die een Tweede Mening Zoekt',
    'Reiziger die Medisch Advies Na Reizen Zoekt',
    'Zorgverlener die Informatie Zoekt']

prompt_factory = PromptFactory(prompt=template,roles=roles)

num_questions_per_doc = 5 

QAGenerator = QAGenerator()
# Generate the data
data = []
for doc in sample:
    try:
        qa_list = QAGenerator.generate_qas(prompt_factory.generate_prompt(doc, num_questions_per_doc))
        for qa in qa_list:
            data.append({'Document': doc, 'Question': qa.question, 'Answer': qa.answer})
    except Exception as e:
            print(f"Failed to generate QA for document: {doc}. Error: {e}")
            continue

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the DataFrame
print(df)


C:\Programming\Automated-QA-Generation\env\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAI instead.
  warn_deprecated(


ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

## Automated Filtering of Q&A Pairs
#### N-gram Similarity

In [ ]:
import pandas as pd
from nltk.util import ngrams
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from ngram_filter import ngram_similarity 

df['Question-Context Similarity'] = df.apply(lambda row: ngram_similarity(row['Question'], row['Document'], 2), axis=1)
df['Answer-Context Similarity'] = df.apply(lambda row: ngram_similarity(row['Answer'], row['Document'], 2), axis=1)

# Display the updated DataFrame
print(df)


## Basic Statistics and Visualization of ngramsimilarity Similarity Scores

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Display basic statistics
print("Basic Statistics for Question-Context Similarity:")
print(df['Question-Context Similarity'].describe())

print("\nBasic Statistics for Answer-Context Similarity:")
print(df['Answer-Context Similarity'].describe())

# Histogram for Question-Context Similarity
plt.figure(figsize=(10, 6))
sns.histplot(df['Question-Context Similarity'], kde=True, bins=20)
plt.title('Distribution of Question-Context Similarity')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.show()

# Histogram for Answer-Context Similarity
plt.figure(figsize=(10, 6))
sns.histplot(df['Answer-Context Similarity'], kde=True, bins=20)
plt.title('Distribution of Answer-Context Similarity')
plt.xlabel('Similarity Score')
plt.ylabel('Frequency')
plt.show()

# Box plots for a side-by-side comparison
plt.figure(figsize=(10, 6))
sns.boxplot(data=df[['Question-Context Similarity', 'Answer-Context Similarity']])
plt.title('Box Plot of Similarity Scores')
plt.ylabel('Similarity Score')
plt.show()


#### Vector similarity

In [ ]:
from vector_filter import precompute_embeddings

embeddings = precompute_embeddings(df, 'Question')


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from vector_filter import create_similarity_matrix


similarity_matrix = create_similarity_matrix(embeddings)

num_rows = similarity_matrix.shape[0]

# Compute minimum similarities for each row, excluding self-similarity
min_similarities = [np.min(row) for i, row in enumerate(similarity_matrix)]
plt.figure(figsize=(10, 6))
plt.hist(min_similarities, bins=20, range=(0, 1))
plt.title('Histogram of Minimum Cosine Similarities')
plt.xlabel('Minimum Cosine Similarity')
plt.ylabel('Number of Rows')
plt.grid(True)
plt.show()

In [ ]:
df


In [ ]:
from vector_filter import filter_dataframe

#df = filter_dataframe(df, embeddings, threshold=0.8)


#### LLM Scoring

In [ ]:
from qa_filtering import estimate_relevance

df['LLM Score'] = df.apply(lambda row: estimate_relevance(row['Question'], row['Document']), axis=1)

plt.figure(figsize=(10, 6))
plt.hist(df['LLM Score'], bins=10, range=(0, 100))
plt.title('Histogram of LLM Scores')
plt.xlabel('LLM Score')
plt.ylabel('Number of Rows')
plt.grid(True)
plt.show()

## Export Q&A Pairs

In [ ]:
df.to_csv('./output.csv')

In [ ]:
df
